# install packages

In [ ]:
pip install nbformat

# execute WalletClustering_AddressIdentification notebook

In [ ]:
%run ./WalletClustering_AddressIdentification.ipynb # includes list creation of addresses
# methods & variables of notebook can be referenced

In [ ]:
# only to remember lists
terrorAddressList # list of extracted flagged terror addresses
blacklistAddresses # list of exchange wallet addresses
addressesToCluster # list of flagged terror addresses without blacklist addresses

# cluster wallets

In [ ]:
#one time change address examples: 3CJNYYeJa4GMN19nLx4rdUjW5hzXXKR8xB,
# attempt to recreate the one time change address heuristic
#WITH * WHERE result = 0
"""
Match (i:Address {isTerrorMultiInput: True})
//There is no address among the outputs that also appears in the inputs (self-change address);
CALL{
    with i
    Match (i:Address ) -[:SENDS]-> (t:Transaction)-[:RECEIVES]->(o:Address)
    with i.address as inputAddresses,o.address as outputAddresses
    return 
    sum(case when inputAddresses in outputAddresses then 1 else 0 end) as result
}
WITH * WHERE result = 0
return i.address
limit 5

"""
""" 
Match (o:Address)
//There is no address among the outputs that also appears in the inputs (self-change address);
CALL{
    with o
    Match (i:address) -[:SENDS]-> (t:Transaction)-[:RECEIVES]->(o)
    with i.address as inputAddresses,o.address as outputAddresses
    return 
    sum(case when inputAddresses in outputAddresses then 1 else 0 end) as frequencyInOutput
}
CALL{
    //Not solved yet
    //This is the first appearance of address O
    //(4) The output addresses other than O do not satisfy condition above.
    with o
    Match   (o)<-[:Receives]-(t:Transaction),
            (a:address)<-[:Receives]-(t)
    return count(t.txid) as firstAppearance, count(a.address) as anotherTransaction
}
WITH * WHERE frequencyInOutput = 0 and firstAppearance = 1 //and anotherTransaction > 1
return o.address
limit 5

//The transaction t is not coin generation; Not finished yet but shouldn't be a problem
Match   (o:Address {address: "112cXA686grivLWLurd1AuAMAghP8hcdjJ"})<-[:RECEIVES]-(t:Transaction),
            (a:Address)<-[:RECEIVES]-(t)
return count(t.txid) as firstAppearance, count(a.address) as anotherTransaction
limit 10

Match (b:Block {hash: "000000000019d6689c085ae165831e934ff763ae46a2a6c172b3f1b60a8ce26f"})
return b
limit 5
"""